# <center> Adherence in a RCT mHealth dataset </center>

### Introduction
Dieses Notebook stellt verschiedene Algorithmen zur Analyse des Adherence Verhaltens von Personen aus einem gegebenen Datensatz zur Verfügung.

### Imports
Damit wir alle benötigten Algorithmen und Funktionen auch aufrufen können, importieren wir zunächst alle Python Dateien, in denen unser Code geschrieben steht.

In [4]:
import sys
sys.path.append('C:/Users/mauri/PycharmProjects/Softwareprojekt/code')
from helper import *
# from task3_prediction import *

### Read in data
Zuerst lesen wir den Datensatz als pandas DataFrame ein. Der Datensatz muss in einem CSV-Format vorliegen. Dafür muss der gesamte Dateipfad zwischen den Hochkommas hinter "dataset =" eingesetzt werden.

In [5]:
dataset = 'C:/Users/mauri/PycharmProjects/Softwareprojekt/data/gefiltert.csv'

# Einlesen des Datensatzes als pandas DataFrame
df = csv_to_dataframe(dataset)

### Identifying Missing Data
Als Nächstes identifizieren wir nicht vorhandene Werte in unserem Datensatz, um sie zu entfernen. So vermeiden wir unnötige Fehlermeldungen aufgrund von ungültigen Werten.

### Group and sort data
Um mit dem Datensatz besser arbeiten zu können müssen wir diesen noch etwas sortieren. Dafür gruppieren wir alle user_ids, sodass die Daten eines Nutzers alle hintereinander stehen und nicht quer verteilt sind und sortieren die Daten nach dem Datum von collected_at, sodass sie auch zeitlich richtig sortiert sind.

In [6]:
# Gruppieren des DataFrame nach user_id und Sortieren nach collected_at
df_sorted = group_and_sort(df)

### Prediction of a users adherence behaviour
In dieser Task geht es darum die Adherence eines neuen Nutzers für einen bestimmten Tag y in der Zukunft vorherzusagen. Dies geschieht mithilfe der Daten die von ihm schon zu Verfügung stehen und mithilfe des Datensatzes. Zuerst lesen wir die Daten unseres Nutzers ein und geben den Tag an, den wir vorhersagen wollen.

In [7]:
new_user = 'C:/Users/mauri/PycharmProjects/Softwareprojekt/data/new_user.csv'
day_y = 20

new_user_df = csv_to_dataframe(new_user)

#### Step 1
Im ersten Schritt suchen wir den Datensatz nach den k-ähnlichsten Personen zu unserem neuen Nutzer ab. Dabei suchen wir nach möglichst ähnlichen Lücken (Anzahl und Länge) im Adherence-Verhalten.

In [5]:
prediction(df_sorted, new_user_df)

          id  user_id locale  \
0       1104     2107     en   
1       1162     2107     de   
2       1217     2107     de   
3       1298     2107     de   
4       1299     2107     de   
...      ...      ...    ...   
17592  69768    40414     es   
17593  69829    40414     es   
17594  70033    40414     es   
17595  70089    40414     es   
17596  70218    40414     es   

                                                  client  \
0      {"name":"UNITI 1.0","os":"iOS 14.5.1","device"...   
1      {"os":"iOS 14.5.1","name":"UNITI 1.0","device"...   
2      {"os":"iOS 14.5.1","name":"UNITI 1.0","device"...   
3      {"os":"iOS 14.5.1","device":"iPhone","name":"U...   
4      {"os":"iOS 14.5.1","device":"iPhone","name":"U...   
...                                                  ...   
17592  {"name":"typ","device":"realme RMX2001","os":"...   
17593  {"name":"typ","device":"realme RMX2001","os":"...   
17594  {"name":"typ","device":"realme RMX2001","os":"...   
17595  {"name":

In [10]:
df_sorted.dtypes

id                          int64
user_id                     int64
locale                     object
client                     object
collected_at               object
created_at                 object
updated_at                 object
value_loudness              int64
value_cumberness            int64
value_jawbone               int64
value_neck                  int64
value_tin_day               int64
value_tin_cumber            int64
value_tin_max               int64
value_movement              int64
value_stress                int64
value_emotion               int64
value_diary_q11            object
collected_at_loudness      object
collected_at_cumberness    object
collected_at_jawbone       object
collected_at_neck          object
collected_at_tin_day       object
collected_at_tin_cumber    object
collected_at_tin_max       object
collected_at_movement      object
collected_at_stress        object
collected_at_emotion       object
collected_at_diary_q11     object
dtype: object

In [17]:
df_sorted['user_id'].unique()

array([ 2107,  2110,  2115,  2118,  2124,  2128,  2130,  2134,  2140,
        2142,  2146,  2149,  2159,  2161,  2165,  2176,  2179,  2183,
        2188,  2191,  2195,  2199,  2204,  2205,  2209,  2214,  2219,
        2222,  2226,  2231,  2234,  2237,  2244,  2251,  2259,  2267,
        2271,  2274,  2280,  2282,  2289,  2294,  2297,  2301,  2308,
        2312,  2317,  2323,  2325,  2329,  2334,  2338,  2342,  2347,
        2351,  2355,  2358,  2367,  2369,  2376,  2378,  2381,  2385,
        2391,  2397,  2402,  2405,  2410,  2415,  2418,  2423,  2429,
        2440,  2442,  2446,  2453,  2461,  2466,  2469,  2475,  2477,
        2482,  2486,  2503, 34107, 34119, 34124, 34126, 34131, 34137,
       34142, 34147, 34150, 34155, 34159, 34162, 34165, 34173, 34179,
       34183, 34187, 34193, 34198, 34204, 34208, 34210, 34218, 34222,
       34227, 34231, 34233, 34242, 34247, 34250, 34255, 34269, 34280,
       34284, 34288, 34292, 34299, 34301, 34306, 34311, 34314, 34317,
       34321, 34325,

In [22]:
len(df_sorted['user_id'].unique())

304